<a href="https://colab.research.google.com/github/Brownwang0426/Reversal-Generative-Reinforcement-Learning/blob/main/train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing requirements (for colab)

In [1]:
!sudo apt-get install python3.10

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.10 is already the newest version (3.10.12-1~22.04.7).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
!pip install pandas==2.0.3 numpy==1.25.2 scipy==1.11.4 swig==4.2.1 ufal.pybox2d==2.3.10.3 gym==0.25.2 pygame==2.5.2 tqdm torch==2.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/

# Cloning git (for colab)

In [15]:
!git clone https://github.com/Brownwang0426/Reversal-Generative-Reinforcement-Learning.git

Cloning into 'Reversal-Generative-Reinforcement-Learning'...
remote: Enumerating objects: 3181, done.
remote: Counting objects: 100% (783/783), done.
remote: Compressing objects: 100% (290/290), done.
remote: Total 3181 (delta 521), reused 706 (delta 489), pack-reused 2398 (from 1)
Receiving objects: 100% (3181/3181), 19.19 MiB | 10.42 MiB/s, done.
Resolving deltas: 100% (2248/2248), done.


# Changing directory (for colab)

In [1]:
import os
os.chdir('/content/Reversal-Generative-Reinforcement-Learning')

# Importing modules

In [2]:
import gym

import numpy as np
import math
from scipy.special import softmax

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, TensorDataset, Subset

import csv

import multiprocessing as mp
import os
import sys
import copy
import random
import gc
import time
from tqdm import tqdm
from collections import defaultdict

import itertools


In [3]:
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Checking cuda

In [4]:
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    device_index = 0
    device = torch.device(f"cuda:{device_index}")
    print('using cuda...')
else:
    device = torch.device("cpu")
    print('using cpu...')
assert device != torch.device("cpu")

Device 0: Tesla T4
using cuda...


In [5]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

# Control board

Crucial configurations regarding how your agent will learn in the environment. The meanings are as follow:
(the configs starting with ⚠️ are what we suggest you must tune according to your specific need in your task)
(the configs starting with ◀️ are what we suggest you to play with to see the effect)

| Configs   | Type   | Description                                                                 |
|------------|--------|-----------------------------------------------------------------------------|
| ⚠️game_name  | STR| The name of the environment.                                |
| ⚠️max_steps_for_each_episode | +INT | The maximun steps that the agent will go through while not done. In some environments, it is crucial to increase your "max_steps_for_each_episode" so that your agent can "live long enough" to obatin some better rewards to gradually and heuristically learn better strategy.                    |
| ◀️ensemble_size  | +INT | The size of the neural ensemble which the agent is comprised of. The bigger, the better, but the longer training time without parallel training. :-D                  |
| ⚠️state_size  | +INT | The size of the state as input data.                    |
| ⚠️hidden_size   | +INT |The size of the hidden layers. We suggest hidden_size >= state_size.           |
| ⚠️action_size   | +INT | The size of action per step as input data.   |
| ⚠️time_size  | +INT |The length of the sequence of actions. Namely, how many steps in the future the agent will predict or use to discern the present best action.                |
| ⚠️reward_size  | +INT |The size of the reward as output data.                          |
| ⚠️neural_type  | STR |  [**`rnn`**, **`gru`**, **`lstm`**, **`rnn_att`**] The type of neural network you prefer. For now, we support rnn, gru, lstm, and rnn_att (recurrent attention). More to come in the future (or you can build one yourself :-D in the models repository).           |
| ⚠️num_layers  | +INT |The number of layers in rnn, gru, lstm, and rnn_att (recurrent attention). We suggest no less than 3 (>= 3) to provide more flexibility and memory capacity for neural networks.                         |
| ⚠️num_heads  | +INT/None |The number of heads in multi-head attention (Should be able to devide hidden_size) (Should be None for non-attention neural_type).                         |
| hidden_activation  | STR | [**`relu`**, **`leaky_relu`**, **`sigmoid`**, **`tanh`**] The type of activation function in the hidden layers.              |
| output_activation  | STR | [**`relu`**, **`leaky_relu`**, **`sigmoid`**, **`tanh`**] The type of activation function in the output layer.                      |
| shift  | 0/±FLOAT |The value in f(x+shift) where f(x) is activation function in the output layer. This value is interesting. If this value is negatively large, the agent will act more conservatively and prone to exploit known strategy. If this value is positively large, the agent to act more radically and prone to explore all possible strategies before settling down.      |
| init   | STR | [**`random_normal`**, **`random_uniform`**, **`xavier_normal`**, **`xavier_uniform`**, **`glorot_normal`**, **`glorot_uniform`**] The initialization method you prefer.                          |
| opti   | STR | [**`adam`**, **`sgd`**, **`rmsprop`**]  The optimization method you prefer.             |
| loss  | STR | [**`mean_squared_error`**, **`binary_crossentropy`**] The loss or error function you prefer.                           |
| bias  | BOLEAN |Whether you want add bias.                          |
| drop_rate   | 0/+FLOAT |The drop-rate for drop-out.              |
| ⚠️alpha   | 0/+FLOAT |The learning rate for neural networks weight matrices.                           |
| ⚠️iteration_for_learning   | +INT |The iteration for learning.              |
| load_pre_model  | BOLEAN |Whether you want to load previous trained model.                          |
| noise_t  |  +INT |The times applying gaussian noise to the initializated actions of the agent, similar to diffusion model's adding gaussian noise.          |
| ⚠️noise_r  |  0/+FLOAT |The noise range to the initializated actions of the agent. The higher the value is, the more exploration-oriented the agent will be.                    |
| ⚠️noise_r_oscillation  |  +INT |The interval for which noise range will oscillate between noise_r and a very small number like 0.000001 to encourage agent to balance eploration and exploitation.                    |
| ⚠️beta  |  0/+FLOAT |The updating rate for updating actions of the agent.              |
| ⚠️iteration_for_deducing  |  +INT |The iteration for updating actions of the agent.                           |
| episode_for_training  | +INT |How many epsiodes will your agent run in the training mode where your agent will learn offline.              |
| chunk_size  | +INT |The maximum chunk size for sequentializing state, action, reward. We suggest chunk_size <= time_size.      |
| batch_size_for_offline_learning  |+INT | After how many epsodes will your agent start learning from experience buffer.                           |
| PER_epsilon  | 0/+FLOAT |The epsilon for prioritized experience replay.              |
| PER_exponent  | 0/+FLOAT |The expoenet for prioritized experience replay.                           |
| episode_for_testing  | +INT |How many epsiodes will your agent run in the testing mode where your agent will not learn offline.                        |
| render_for_human  | BOLEAN | Wether you want to render the visual result for each step in the testing mode.              |


## frozen lake

In [6]:
game_name =  'FrozenLake-v1'        #⚠️  gym.make(game_name, is_slippery=False, map_name="4x4")
max_steps_for_each_episode = 25     #⚠️


ensemble_size = 5                   #◀️
state_size =  16                    #⚠️
hidden_size = 100                   #⚠️
action_size = 4                     #⚠️
time_size = 8                       #⚠️
reward_size = 100                   #⚠️
neural_type = 'gru'                 #⚠️
num_layers = 3                      #⚠️
num_heads = None                    #⚠️
hidden_activation = 'tanh'
output_activation = 'sigmoid'
shift = 0.0
init = "random_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                         #⚠️
iteration_for_learning = 1000       #⚠️
load_pre_model = False


noise_t = 1                         #⚠️
noise_r = 0.1                       #⚠️
noise_r_oscillation = 10            #⚠️
beta = 0.1                          #⚠️
iteration_for_deducing = 100        #⚠️


episode_for_training = 100000
chunk_size = time_size
batch_size_for_offline_learning = 1
PER_epsilon = 0.000001
PER_exponent = 5


episode_for_testing = 100
render_for_human = True



## blackjack

In [7]:
game_name = 'Blackjack-v1'          #⚠️
max_steps_for_each_episode = 10     #⚠️


ensemble_size = 5                   #◀️
state_size =  201                   #⚠️
hidden_size = 250                   #⚠️
action_size = 2                     #⚠️
time_size = 5                       #⚠️
reward_size = 100                   #⚠️
neural_type = 'gru'                 #⚠️
num_layers = 3                      #⚠️
num_heads = None                    #⚠️
hidden_activation = 'tanh'
output_activation = 'sigmoid'
shift = 0.0
init = "random_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                         #⚠️
iteration_for_learning = 1000       #⚠️
load_pre_model = False


noise_t = 1
noise_r = 0.1                       #⚠️
noise_r_oscillation = 10            #⚠️
beta = 0.1                          #⚠️
iteration_for_deducing = 100        #⚠️


episode_for_training = 100000
chunk_size = time_size
batch_size_for_offline_learning = 1
PER_epsilon = 0.000001
PER_exponent = 5


episode_for_testing = 100
render_for_human = True


## cartpole

In [8]:
game_name = 'CartPole-v1'           #⚠️
max_steps_for_each_episode = 2000   #⚠️


ensemble_size = 10                  #◀️
state_size =  400                   #⚠️
hidden_size = 400                   #⚠️
action_size = 2                     #⚠️
time_size = 25                      #⚠️
reward_size = 100                   #⚠️
neural_type = 'gru'                 #⚠️
num_layers = 3                      #⚠️
num_heads = None                    #⚠️
hidden_activation = 'tanh'
output_activation = 'sigmoid'
shift = 0.0
init = "random_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                         #⚠️
iteration_for_learning = 1000       #⚠️
load_pre_model = False


noise_t = 1                         #⚠️
noise_r = 0.1                       #⚠️
noise_r_oscillation = 10            #⚠️
beta = 0.1                          #⚠️
iteration_for_deducing = 100        #⚠️


episode_for_training = 100000
chunk_size = time_size
batch_size_for_offline_learning = 1
PER_epsilon = 0.000001
PER_exponent = 5


episode_for_testing = 100
render_for_human = True


## mountain car

In [9]:
game_name =  'MountainCar-v0'       #⚠️
max_steps_for_each_episode = 200    #⚠️


ensemble_size = 10                  #◀️
state_size =  200                   #⚠️
hidden_size = 200                   #⚠️
action_size = 3                     #⚠️
time_size = 50                      #⚠️
reward_size = 100                   #⚠️
neural_type = 'gru'                 #⚠️
num_layers = 3                      #⚠️
num_heads = None                    #⚠️
hidden_activation = 'tanh'
output_activation = 'sigmoid'
shift = 0.0
init = "random_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                         #⚠️
iteration_for_learning = 1000       #⚠️
load_pre_model = False


noise_t = 1
noise_r = 0.1                       #⚠️
noise_r_oscillation = 10            #⚠️
beta = 0.1                          #⚠️
iteration_for_deducing = 100        #⚠️


episode_for_training = 100000
chunk_size = time_size
batch_size_for_offline_learning = 1
PER_epsilon = 0.000001
PER_exponent = 5


episode_for_testing = 100
render_for_human = True


## acrobot

In [10]:
game_name = 'Acrobot-v1'            #⚠️
max_steps_for_each_episode = 200    #⚠️


ensemble_size = 5                   #◀️
state_size =  600                   #⚠️
hidden_size = 600                   #⚠️
action_size = 3                     #⚠️
time_size = 50                      #⚠️
reward_size = 100                   #⚠️
neural_type = 'gru'                 #⚠️
num_layers = 3                      #⚠️
num_heads = None                    #⚠️
hidden_activation = 'tanh'
output_activation = 'sigmoid'
shift = 0.0
init = "random_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                         #⚠️
iteration_for_learning = 1000       #⚠️
load_pre_model = False


noise_t = 1
noise_r = 0.1                       #⚠️
noise_r_oscillation = 10            #⚠️
beta = 0.1                          #⚠️
iteration_for_deducing = 100        #⚠️


episode_for_training = 100000
chunk_size = time_size
batch_size_for_offline_learning = 1
PER_epsilon = 0.000001
PER_exponent = 5


episode_for_testing = 100
render_for_human = True


## lunar lander

In [11]:
game_name = "LunarLander-v2"        #⚠️
max_steps_for_each_episode = 200    #⚠️


ensemble_size = 5                   #◀️
state_size =  800                   #⚠️
hidden_size = 800                   #⚠️
action_size = 4                     #⚠️
time_size = 50                      #⚠️
reward_size = 250                   #⚠️
neural_type = 'gru'                 #⚠️
num_layers = 3                      #⚠️
num_heads = None                    #⚠️
hidden_activation = 'tanh'
output_activation = 'sigmoid'
shift = 0.0
init = "random_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                         #⚠️
iteration_for_learning = 1000       #⚠️
load_pre_model = False


noise_t = 1                         #⚠️
noise_r = 0.1                       #⚠️
noise_r_oscillation = 10            #⚠️
beta = 0.1                          #⚠️
iteration_for_deducing = 100        #⚠️


episode_for_training = 100000
chunk_size = time_size
batch_size_for_offline_learning = 1
PER_epsilon = 0.000001
PER_exponent = 5


episode_for_testing = 100
render_for_human = True



## your present config

In [12]:
game_name =  'FrozenLake-v1'        #⚠️
max_steps_for_each_episode = 25     #⚠️


ensemble_size = 5                   #◀️
state_size =  16                    #⚠️
hidden_size = 100                   #⚠️
action_size = 4                     #⚠️
time_size = 8                       #⚠️
reward_size = 100                   #⚠️
neural_type = 'gru'                 #⚠️
num_layers = 3                      #⚠️
num_heads = None                    #⚠️
hidden_activation = 'tanh'
output_activation = 'sigmoid'
shift = 0.0
init = "random_normal"
opti = 'sgd'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.1                         #⚠️
iteration_for_learning = 1000       #⚠️
load_pre_model = False


noise_t = 1                         #⚠️
noise_r = 0.1                       #⚠️
noise_r_oscillation = 10            #⚠️
beta = 0.1                          #⚠️
iteration_for_deducing = 100        #⚠️


episode_for_training = 100000
chunk_size = time_size
batch_size_for_offline_learning = 1
PER_epsilon = 0.000001
PER_exponent = 5


episode_for_testing = 100
render_for_human = True



In [13]:
suffix                      = f"game={game_name}_type={neural_type}_ensemble={ensemble_size:05d}_drop={drop_rate:.5f}_learn={iteration_for_learning:05d}_interval={batch_size_for_offline_learning:05d}_deduce={iteration_for_deducing:05d}"
directory                   = f'./result/{game_name}/'
model_directory             = f'./result/{game_name}/model_{suffix}'+'_%s.h5'
performance_log_directory   = f'./result/{game_name}/performace_log_{suffix}.csv'

# Importing local modules

In [14]:
if   game_name == 'FrozenLake-v1':
    from envs.env_frozenlake   import vectorizing_state, vectorizing_action, vectorizing_reward
elif   game_name == 'Blackjack-v1':
    from envs.env_blackjack   import vectorizing_state, vectorizing_action, vectorizing_reward
elif   game_name == 'CartPole-v1':
    from envs.env_cartpole    import vectorizing_state, vectorizing_action, vectorizing_reward
elif game_name == 'MountainCar-v0':
    from envs.env_mountaincar import vectorizing_state, vectorizing_action, vectorizing_reward
elif game_name == 'MountainCarContinuous-v0':
    from envs.env_mountaincar_continuous import vectorizing_state, vectorizing_action, vectorizing_reward
elif game_name == 'Acrobot-v1':
    from envs.env_acrobot import vectorizing_state, vectorizing_action, vectorizing_reward
elif game_name == "Pendulum-v1":
    from envs.env_pendulum import vectorizing_state, vectorizing_action, vectorizing_reward
elif game_name == "LunarLander-v2":
    from envs.env_lunarlander import vectorizing_state, vectorizing_action, vectorizing_reward
elif game_name == 'BipedalWalker-v3':
    from envs.env_bipedalwalker import vectorizing_state, vectorizing_action, vectorizing_reward
else:
   raise RuntimeError('missing env functions')

In [15]:
if neural_type == 'rnn_att':
    from models.model_rnn_att import build_model
    from utils.util_rnn_att   import initialize_pre_activated_actions, \
                                 update_pre_activated_actions, \
                                 sequentialize, \
                                 update_model,\
                                 save_performance_to_csv
else:
    from models.model_rnn import build_model
    from utils.util_rnn_  import initialize_pre_activated_actions, \
                                 update_pre_activated_actions, \
                                 sequentialize, \
                                 update_model,\
                                 save_performance_to_csv

# Deducing -> Learning
Training mode where your agent will learn offline. You can see here how your agent learn overtime and improve its performance.

Creating or loading models

In [16]:

if not os.path.exists(directory):
    os.makedirs(directory)

if load_pre_model == False:

    model_list = []
    for _ in range(ensemble_size):
        model = build_model(state_size,
                            hidden_size,
                            action_size,
                            time_size,
                            reward_size,
                            neural_type,
                            num_layers,
                            num_heads,
                            hidden_activation,
                            output_activation,
                            shift,
                            init,
                            opti,
                            loss,
                            bias,
                            drop_rate,
                            alpha)
        model.to(device)
        model_list.append(model)

elif load_pre_model == True:

    model_list = []
    for _ in range(ensemble_size):
        model = build_model(state_size,
                            hidden_size,
                            action_size,
                            time_size,
                            reward_size,
                            neural_type,
                            num_layers,
                            num_heads,
                            hidden_activation,
                            output_activation,
                            shift,
                            init,
                            opti,
                            loss,
                            bias,
                            drop_rate,
                            alpha)
        model.to(device)
        model_list.append(model)

    for i in range(len(model_list)):
        model_list[i].load_state_dict(torch.load( model_directory  % i ))



Creating Streams

In [17]:
stream_list = []
for _ in range(ensemble_size):
    stream  = torch.cuda.Stream()
    stream_list.append(stream)

Creating desired reward

In [18]:
desired_reward = torch.ones((1, time_size, reward_size))

Putting all the previous works into play

In [19]:

performance_log  = []

for training_episode in tqdm(range(episode_for_training)):

    # initializing short term experience replay buffer
    list_states  = []
    list_actions = []
    list_rewards = []

    # initializing environment
    env                    = gym.make(game_name, is_slippery=False, map_name="4x4")
    env._max_episode_steps = max_steps_for_each_episode
    state                  = env.reset()
    summed_reward          = 0

    # observing state
    state    = vectorizing_state(state)
    list_states.append(state)

    for count in itertools.count(1):

        print(f'\rStep: {count}\r', end='', flush=True)

        # initializing and updating action
        state                 = torch.tensor(np.atleast_2d(state), dtype=torch.float)
        t_oscillation         =  (noise_t if (training_episode % (2 * noise_r_oscillation) < noise_r_oscillation) else 1)
        r_oscillation         =  (noise_r if (training_episode % (2 * noise_r_oscillation) < noise_r_oscillation) else 0.000001)
        pre_activated_action  = initialize_pre_activated_actions(init,
                                                                t_oscillation,
                                                                r_oscillation,
                                                                (time_size, action_size))
        pre_activated_action  = torch.tensor(pre_activated_action[np.newaxis, :, :], dtype=torch.float)
        pre_activated_action  = update_pre_activated_actions(iteration_for_deducing,
                                                             model_list,
                                                             state,
                                                             pre_activated_action,
                                                             desired_reward,
                                                             beta,
                                                             device)
        action, action_       = vectorizing_action(pre_activated_action)
        list_actions.append(action)

        # executing action
        state, reward, done, info = env.step(action_)

        # observing actual reward
        summed_reward += reward
        reward = vectorizing_reward(state, reward, summed_reward, done, reward_size)
        list_rewards.append(reward)

        # observing state
        state    = vectorizing_state(state)
        list_states.append(state)

        if done:
            print(f'Episode {training_episode}: Summed_Reward = {summed_reward}')
            performance_log.append([training_episode, summed_reward])
            save_performance_to_csv(performance_log, performance_log_directory)
            break
        else:
            pass

    env.close()




    # sequentializing short term experience replay buffer
    present_state_tensors   ,\
    future_actions_tensors  ,\
    future_rewards_tensors   ,\
    future_states_tensors    ,\
    pad_size_tensors            = sequentialize(list_states  ,
                                            list_actions ,
                                            list_rewards , chunk_size, device)

    if training_episode == 0:
      long_term_present_state_tensors   = copy.deepcopy(present_state_tensors  )
      long_term_future_actions_tensors  = copy.deepcopy(future_actions_tensors )
      long_term_future_rewards_tensors   = copy.deepcopy(future_rewards_tensors  )
      long_term_future_states_tensors    = copy.deepcopy(future_states_tensors   )
      long_term_pad_size_tensors            = copy.deepcopy(pad_size_tensors           )

      # storing sequentialized short term experience to long term experience replay buffer by length when it is new
      existing_hashes = set(
          hash((tuple(t.numpy().flatten()) for t in tensors))
          for tensors in zip(
              long_term_present_state_tensors,
              long_term_future_actions_tensors,
              long_term_future_rewards_tensors,
              long_term_future_states_tensors,
              long_term_pad_size_tensors,
          )
      )
    else:
      for i in range(len(present_state_tensors)):
          new_sample = (
              present_state_tensors[i],
              future_actions_tensors[i],
              future_rewards_tensors[i],
              future_states_tensors[i],
              pad_size_tensors[i]
          )
          sample_hash = hash(tuple(t.numpy().flatten()) for t in new_sample)
          if sample_hash not in existing_hashes:

              long_term_present_state_tensors  = torch.cat((long_term_present_state_tensors  , new_sample[0].unsqueeze(0)), dim=0)
              long_term_future_actions_tensors = torch.cat((long_term_future_actions_tensors , new_sample[1].unsqueeze(0)), dim=0)
              long_term_future_rewards_tensors  = torch.cat((long_term_future_rewards_tensors  , new_sample[2].unsqueeze(0)), dim=0)
              long_term_future_states_tensors   = torch.cat((long_term_future_states_tensors   , new_sample[3].unsqueeze(0)), dim=0)
              long_term_pad_size_tensors           = torch.cat((long_term_pad_size_tensors           , new_sample[4].unsqueeze(0)), dim=0)

              existing_hashes.add(sample_hash)




    # batch offline learning
    if (training_episode+1) % batch_size_for_offline_learning == 0:


        start_time = time.time()


        # training with Prioritized Experience Replay (PER)
        for i, model in enumerate(model_list):
            with torch.cuda.stream(stream_list[i]):
                model                     = update_model(iteration_for_learning,
                                                         long_term_present_state_tensors  ,
                                                         long_term_future_actions_tensors ,
                                                         long_term_future_rewards_tensors  ,
                                                         long_term_future_states_tensors   ,
                                                         long_term_pad_size_tensors           ,
                                                         model,
                                                         PER_epsilon,
                                                         PER_exponent,
                                                         device)
                model_list[i]             = model
        torch.cuda.synchronize()


        end_time = time.time()  # Record end time
        execution_time = end_time - start_time  # Calculate duration
        print(f"Execution Time: {execution_time:.4f} seconds")


        # saving:
        for i in range(len(model_list)):
            torch.save(model_list[i].state_dict(), model_directory % i)


        gc.collect()
        torch.cuda.empty_cache()

  0%|          | 0/100000 [00:00<?, ?it/s]

Episode 0: Summed_Reward = 0.0
tensor([1, 1, 1, 2, 2, 3], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3], device='cuda:0')


  0%|          | 1/100000 [02:07<3531:55:26, 127.15s/it]

Execution Time: 123.0216 seconds
Episode 1: Summed_Reward = 0.0
tensor([1, 1, 1, 2, 2, 3, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3, 1], device='cuda:0')


  0%|          | 2/100000 [04:18<3597:09:38, 129.50s/it]

Execution Time: 126.1485 seconds
Episode 2: Summed_Reward = 0.0
tensor([1, 1, 1, 2, 2, 3, 1, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3, 1, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3, 1, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3, 1, 1], device='cuda:0')
tensor([1, 1, 1, 2, 2, 3, 1, 1], device='cuda:0')


  0%|          | 3/100000 [06:33<3665:10:30, 131.95s/it]

Execution Time: 131.6154 seconds
Episode 3: Summed_Reward = 0.0
tensor([1, 1, 1, 2, 2, 3, 1, 1, 1], device='cuda:0')


  0%|          | 3/100000 [06:59<3882:23:44, 139.77s/it]


KeyboardInterrupt: 

# Deducing only
Testing mode where your trained agent in the training mode will not learn offline. It just keeps running each episode without learning new stuff.

Loading models

In [ ]:
model_list = []
for _ in range(ensemble_size):
    model = build_model(state_size,
                        hidden_size,
                        action_size,
                        time_size,
                        reward_size,
                        neural_type,
                        num_layers,
                        num_heads,
                        hidden_activation,
                        output_activation,
                        shift,
                        init,
                        opti,
                        loss,
                        bias,
                        drop_rate,
                        alpha)
    model.to(device)
    model_list.append(model)

for i in range(len(model_list)):
    model_list[i].load_state_dict(torch.load(model_directory % i))

Creating desired reward ... again

In [ ]:
desired_reward = torch.ones((1, time_size, reward_size))

Putting all the previous works into play ... again

But this time the agent does not learn

In [ ]:
total_summed_reward = 0

for testing_episode in range(episode_for_testing):

    if render_for_human == True:
        env = gym.make( game_name, render_mode="human")
    else:
        env = gym.make( game_name)
    env._max_episode_steps = max_steps_for_each_episode
    state                  = env.reset()
    if render_for_human == True:
        env.render()
    summed_reward = 0

    state = vectorizing_state(state)

    done = False
    while not done:

        state                 = torch.tensor(np.atleast_2d(state), dtype=torch.float)
        pre_activated_action  = initialize_pre_activated_actions(init, noise_t, noise_r, (time_size, action_size))
        pre_activated_action  = torch.tensor(pre_activated_action[np.newaxis, :, :], dtype=torch.float)
        pre_activated_action  = update_pre_activated_actions(iteration_for_deducing,
                                                             model_list,
                                                             state,
                                                             pre_activated_action,
                                                             desired_reward,
                                                             beta,
                                                             device)
        action, action_       = vectorizing_action(pre_activated_action)

        state, reward, done,  info = env.step(action_)
        if render_for_human == True:
            env.render()

        summed_reward += reward

        state = vectorizing_state(state)

        if done:
            break

    env.close()

    print("Summed reward:", summed_reward)
    print(f'Episode: {testing_episode + 1}')
    print('Everaged summed reward:')
    total_summed_reward += summed_reward
    print(total_summed_reward/(testing_episode + 1))



In [ ]:
import torch

# Input tensor
input_tensor = torch.tensor([4, 2, 3])  # Shape: (3,)

# Fixed length for each row
fixed_length = 50

# Generate a range for the fixed length
range_tensor = torch.arange(fixed_length)  # Shape: (fixed_length,)
print(range_tensor)
# Compare and create the mask
output_tensor = (range_tensor < input_tensor.unsqueeze(1)).int()  # Shape: (3, fixed_length)

print(output_tensor)


In [ ]:
import torch

# Example tensors
tensor_3d = torch.randn(2, 3, 999)  # Shape: (batch_size, num_rows, feature_size)
print(tensor_3d)
tensor_2d = torch.tensor([[1, 2, 3], [4, 5, 6]])  # Shape: (batch_size, num_rows)
print(tensor_2d)
# Expand the 2D tensor to match the 3D tensor's shape for broadcasting
tensor_2d_expanded = tensor_2d.unsqueeze(-1)  # Shape: (batch_size, num_rows, 1)

# Perform element-wise multiplication
result = tensor_3d * tensor_2d_expanded
print(result)
print(f"3D Tensor Shape: {tensor_3d.shape}")
print(f"2D Tensor Shape: {tensor_2d.shape}")
print(f"Expanded 2D Tensor Shape: {tensor_2d_expanded.shape}")
print(f"Result Shape: {result.shape}")
